In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import db_connection
import db_methods

In [2]:
MongoDB connection
channels_data_collection = db_connection.connect_to_mongodb("Channel Data")
channels_data = db_methods.retrieve_all_channels_from_mongodb(channels_data_collection)

# I. Channels Dataset Data Cleaning

The following steps will focus on data preprocessing of the dataset Channels to prepare it for further analysis.

In [6]:
cleaned_channels = channels_data.copy()

First let's see the information about the dataset, is there some missing values.

In [7]:
cleaned_channels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156977 entries, 0 to 156976
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   category_cc         156846 non-null  object 
 1   join_date           156976 non-null  object 
 2   channel             156977 non-null  object 
 3   name_cc             156965 non-null  object 
 4   subscribers_cc      156977 non-null  int64  
 5   videos_cc           156977 non-null  int64  
 6   subscriber_rank_sb  153330 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 8.4+ MB


As we can see here, the columns **category_cc**, **join_date**, **name_cc**, and **subscriber_rank_sb** have missing values. The **subscriber_rank_sb** column is of type float64 instead of int64. We should also consider converting the **join_date** column to a pd.datetime type.

### 1. join_date column

In [9]:
    # Show to lines with missing value of join_data
cleaned_channels[cleaned_channels['join_date'].isna()]

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb
33540,Entertainment,NaN,UCBR8-60-B28hp2BmDPdntcQ,YouTube,28200000,301,110.0


There is only one missing value, so we can drop it.

In [10]:
cleaned_channels.drop(33540, inplace=True)

### 2. name_cc column

In [13]:
    # Show to lines with missing value of name_cc
cleaned_channels[cleaned_channels['name_cc'].isna()]

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb
8231,Entertainment,2014-01-01,UCMzbrcOG-edOxonhLbsOD4Q,NaN,112543,118,140756.0
8232,Gaming,2015-02-01,UCJIYgIBaQt0YcGO73VCLPyg,NaN,57000,180,266879.0
8233,Entertainment,2007-03-27,UCp5Tpg0lLGtsG88CvUKey_A,NaN,12400,362,789457.0
8234,Entertainment,2016-07-09,UCmOiT9ZnSvLZ1XMBFjGiNmQ,NaN,55100,26,249430.0
8235,Film and Animation,2016-09-27,UChiZ8VX_WjTGvhpxHub16_A,NaN,31600,55,349306.0
8236,Entertainment,2009-03-12,UCgtijEzltp6pTR6ezYCwUQQ,NaN,16500,59,662889.0
8237,Gaming,2016-05-26,UCv_OpX7RULreDQOhZlVLuUw,NaN,11000,108,929597.0
8238,Music,2011-12-31,UCA67sOj4L0qlqOHQZsmUhVw,NaN,15200,68,736183.0
8239,Entertainment,2007-12-04,UCYj8nTwafpwx0T8wKkWuwag,NaN,42000,10,338910.0
8240,Music,2011-08-26,UCq-68gzHO5ulmrZ2tW-jyTg,NaN,19800,390,535474.0


There are 12 rows with channels that don't have a name, so we can drop them.

In [14]:
cleaned_channels['name_cc'].fillna('unknown name', inplace=True)

### 3. category_cc column

In [17]:
    # Show to lines with missing value of category_cc
cleaned_channels[cleaned_channels['category_cc'].isna()]

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb
3100,NaN,2011-11-10,UCUC0f5Lu42bIi9IClnYP4zw,PixarMoviesOnDemand,138000,16,126929.0
5002,NaN,2006-04-08,UCJdx55eTMBJI9dwuP65xcSg,nevake,28600,9168,444224.0
5113,NaN,2016-07-24,UCfL3FVHRncVVoBnVI_EI8vw,spoiled milk,10600,33,938514.0
5143,NaN,2014-02-17,UCDhrs5GhG2t1Ov3oya3K4tg,TwitchPlaysPokemon,10300,342,977938.0
5638,NaN,2007-05-29,UCBgt-JTMivpgj-sHaea6M-Q,New Music,22800,398,507809.0
...,...,...,...,...,...,...,...
146651,NaN,2013-04-09,UC-jhZdJL4gDmATXCqMkM7pA,EtherealExposition,16435,63,631214.0
149433,NaN,2006-05-12,UCVEfyEuLJpgw_y4jSgV8Rtg,lonelygirl15,173903,396,100037.0
150554,NaN,2011-06-14,UC_3JbOmL48TMal015yX13Iw,FyreUK [ARCHIVE],303000,466,60223.0
151278,NaN,2008-11-17,UCEPE1gwLMgNmTFQSSfXAIcg,Danielle Lowe,36300,78,346694.0


As we can see, there are 131 rows without a category. Let's investigate what percentage this represents.

In [87]:
print(f'the channels with unknown category represent : {round(13100/len(channels_data), 3)}%')

the channels with unknown category represent : 0.083%


The **channels with unknown category** represent **less than 0.01%** of the total data, we can drop them.

In [20]:
cleaned_channels = cleaned_channels[cleaned_channels['category_cc'].notna()]

### 4. subscriber_rank_sb column

In [23]:
    # Show to lines with missing value of subscribers_cc
cleaned_channels.sort_values(by='subscribers_cc', ascending=False)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb
29058,Music,2006-03-13,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,112139463,13839,102.0
16032,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0
43231,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0
35502,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0
40878,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0
...,...,...,...,...,...,...,...
80084,Music,2017-07-27,UCbKqo5ccPuQad_17MoSxpvw,Ayanami,10000,20,876195.0
152522,Entertainment,2014-05-02,UC-AKZXBB30O4ZM5zVLGUPCg,ArabianRider,10000,212,977030.0
118797,Music,2011-08-27,UCVXWrVUv_Pbt24btuuIDvqw,Densle,10000,60,944207.0
13068,Music,2016-04-30,UCu4TumUmUdU_YmbSuvEOvNg,Trailer Music Empire...,10000,157,812348.0


The **subscriber_rank_sb** feature is incorrect, as we can se it above. We can recrate this column.

In [25]:
cleaned_channels['subscriber_rank_sb'] = cleaned_channels['subscribers_cc'].rank(ascending=False, method='dense')

In [26]:
cleaned_channels.sort_values(by = 'subscribers_cc', ascending=False)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb
29058,Music,2006-03-13,UCq-Fj5jknLsUf-MWSy4_brA,T-Series,112139463,13839,1.0
16032,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,2.0
43231,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,3.0
35502,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,4.0
40878,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,5.0
...,...,...,...,...,...,...,...
80084,Music,2017-07-27,UCbKqo5ccPuQad_17MoSxpvw,Ayanami,10000,20,31237.0
152522,Entertainment,2014-05-02,UC-AKZXBB30O4ZM5zVLGUPCg,ArabianRider,10000,212,31237.0
118797,Music,2011-08-27,UCVXWrVUv_Pbt24btuuIDvqw,Densle,10000,60,31237.0
13068,Music,2016-04-30,UCu4TumUmUdU_YmbSuvEOvNg,Trailer Music Empire...,10000,157,31237.0


The problem is fixed

### 5. Type conversion column

The column **subscriber_rank_sb** is of the wrong type, we can convert it to int64.

In [30]:
cleaned_channels['subscriber_rank_sb'] = cleaned_channels['subscriber_rank_sb'].astype('int64')

We can convert the **datetime** column to pd.datetime

In [33]:
datetime_column = pd.to_datetime(cleaned_channels['join_date'])

We can also create new columns for the year, the month and the day of creation of the channel.

In [ ]:
cleaned_channels['year'] = datetime_column.dt.year
cleaned_channels['month'] = datetime_column.dt.month
cleaned_channels['day'] = datetime_column.dt.day

### 6. Final Dataset

In [96]:
cleaned_channels.head()

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,year,month,day
0,Film and Animation,2017-05-21,UCBJuEqXfXTdcPSbGO9qqn1g,MagnusNation,65100,28,15856,2017,5,21
1,Entertainment,2011-12-13,UCkNW9Q1VR_aeZ6uht83jJVQ,Mago Dario Animazion...,60200,48,16388,2011,12,13
2,Music,2008-03-17,UCXhkGgooXHDNwgJXmoTSN7g,Mago Merlino,14800,838,27869,2008,3,17
3,Entertainment,2014-10-19,UCvZGsuvKlYOGiZTsxwJNS5Q,MAGO TOMÁS,26200,31,22950,2014,10,19
4,Comedy,2017-01-15,UCxJWPpPED-J24znoKyKZYjg,Magog of Morskar,29400,158,21938,2017,1,15


In [99]:
cleaned_channels.info()

<class 'pandas.core.frame.DataFrame'>
Index: 156845 entries, 0 to 156976
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   category_cc         156845 non-null  object
 1   join_date           156845 non-null  object
 2   channel             156845 non-null  object
 3   name_cc             156845 non-null  object
 4   subscribers_cc      156845 non-null  int64 
 5   videos_cc           156845 non-null  int64 
 6   subscriber_rank_sb  156845 non-null  int64 
 7   year                156845 non-null  int32 
 8   month               156845 non-null  int32 
 9   day                 156845 non-null  int32 
dtypes: int32(3), int64(3), object(4)
memory usage: 11.4+ MB


### 7. Full cleaning pipeline

After testing various preprocessing steps, we've combined them into a single cleaning pipeline that performs all the necessary tasks, including handling missing data, removing irrelevant rows, ranking channels by subscriber count, and creating date features.

In [37]:
def cleaning_pipeline(data):

    cleaned_data = data.copy()
    
        # delete the line with joint_data = NaN
    cleaned_data.drop(33540, inplace = True)

        # remplace name_cc that are NaN by 'unknown name'
    cleaned_data['name_cc'].fillna('unknown name', inplace=True)

        # drop channels with unknown category
    cleaned_data = cleaned_data[cleaned_data['category_cc'].notna()]

        # recreate the subscriber_rank_sb
    cleaned_data['subscriber_rank_sb'] = cleaned_data['subscribers_cc'].rank(ascending=False, method='dense')

        # subscriber_rank_sb type conversion
    cleaned_data['subscriber_rank_sb'] = cleaned_data['subscriber_rank_sb'].astype('int64')

        # date feature creation
    datetime_column = pd.to_datetime(cleaned_data['join_date'])
    cleaned_data['year'] = datetime_column.dt.year
    cleaned_data['month'] = datetime_column.dt.month
    cleaned_data['day'] = datetime_column.dt.day

    return cleaned_data
    

## II - Timeseries Dataset Data Cleaning

The following steps will focus on data preprocessing of the dataset time_series to prepare it for further analysis.

In [40]:
print("Loading data...")
data_path = ''
time_series = pd.read_csv(data_path + '\df_timeseries.tsv', sep='\t')

Loading data...


In [42]:
MongoDB connection
channels_weekly_time_series = db_connection.connect_to_mongodb("Channels Weekly Time Series")
channels_time_series  = db_methods.retrieve_all_channels_from_mongodb(channels_weekly_time_series)

### 1. Handling missing values

time_series.info()

Since the dataset is to big, we can't see wich column have missing value, let's try another thing.

In [44]:
col_with_missing_values = [col for col in time_series.columns if time_series[col].isnull().sum() > 0]

print("Columns with missing values :", col_with_missing_values)

Columns with missing values : ['category']


Only the category column has missing values. We can investigate what percentage of values are missing.

In [45]:
number_of_missing_categories = time_series['category'].isna().sum()
print(f'{ 100 * number_of_missing_categories/len(time_series)} % of categories values are missing')

0.09709725240015055 % of categories values are missing


Less than **0.1%** of channels have unknown category, we can drop them.

In [47]:
cleaned_time_series = time_series[time_series['category'].notna()]

It was the only problem of this dataset, we can also convert the datetime column to pd.datetime

We can create a pipeline that performs cleaning and data conversion.

### 2.Full cleaning Pipeline

We can combine these two steps into a cleaning pipeline.

In [49]:

def time_series_cleaning_pipeline(data):

    data = data.copy()
        # Remove the lines with unknown category
    data = data[data['category'].notna()]
        # turn the datatime column to a pd.datetime
    data['datetime'] = pd.to_datetime(data['datetime'])

    return data
    